### Importing Packages

In [31]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
CLIENT_ID = 'TJ0D0H1JMZCX3N1YYXGL0D3VSNIOHFO1DSNFW3ZIWIPB4Q2N' # your Foursquare ID
CLIENT_SECRET = 'ODLIF4YATNNGVV0FMCPA1KEIKR1GZUPZGYRPWSENJ5DNHWIX' # your Foursquare Secret
VERSION = '201905018'

### Parsing Data from Table into DataFrame

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
site_content = BeautifulSoup(page.content, 'lxml').find('table', class_ = 'wikitable sortable')

In [3]:
table_rows = site_content.find_all('tr')

array = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        array.append(row)

In [4]:
df = pd.DataFrame(array, columns = ['PostalCode','Borough','Neighborhood'])
df_2 = df[df.Borough != 'Not assigned']
df_3 = df_2.groupby(['PostalCode','Borough'], sort = True)['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
print(df_3.shape)
df_3.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df_4 = pd.read_csv("http://cocl.us/Geospatial_data")
del df_4['Postal Code']
df_5 = df_3.join(df_4)
df_5.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Appending Latitudes and Longitudes of Postal Codes to Dataframe

In [6]:
print(df_5.Borough.value_counts())
print("There are a total of 38 boroughs in Toronto containing the word \'Toronto'")

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64
There are a total of 38 boroughs in Toronto containing the word 'Toronto'


### Creating New Dataframe Consisting of Boroughs Containing the Word "Toronto" and Mapping Postal Codes

In [7]:
#New dataframe in which 'Borough' column only contains boroughs with 'Toronto' in the name
toronto_data = df_5[df_5['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
toronto_latitude = 43.6532
toronto_longitude = -79.3832
#map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)
#map_toronto

#for i in range(0,len(toronto_data)):
 #   folium.Marker([toronto_data.iloc[i]['Latitude'], toronto_data.iloc[i]['Longitude']], popup=toronto_data.iloc[i]['Neighborhood']).add_to(map_toronto)
#map_toronto


map_toronto2 = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)
map_toronto2

for lat, lng, Borough, Neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(Borough, Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='coral',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2


In [9]:
toronto_data.dtypes

PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object